## Predicción de Personalidad - Imputación Avanzada y Optimización con Optuna

#### **Parte 2**

Fecha: 28 julio 2025

**Autor: Ricardo Urdaneta**

Este notebook es la continuación del análisis inicial. Partiendo de un modelo robusto que alcanzó un score de **0.973279**, el objetivo aquí es explorar si técnicas más sofisticadas de preprocesamiento y optimización pueden superar el benchmark establecido.

#### Metodología Avanzada

Preprocesamiento avanzado utilizando **`IterativeImputer`** de Scikit-learn, un método multivariado que estima cada valor faltante basándose en las demás características.

Optimización de hiperparámetros con **`Optuna`** para realizar una búsqueda más inteligente y eficiente del mejor set de parámetros para RandomForestClassifier y XGBoost.

Comparación de modelos de ensamble, incluyendo **`VotingClassifier`** y **`StackingClassifier`**, para encontrar la mejor estrategia de combinación de modelos.

In [ ]:
# Para manipulación de datos
import pandas as pd
import numpy as np

# Para preprocesamiento y modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Es necesario importar esto para habilitar IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#### Imputación con IterativeImputer

In [ ]:
#  PREPROCESAMIENTO 

# Cargar y preprocesamiento inicial
df_train = pd.read_csv('train.csv')
df_processed = df_train.copy()
df_processed = df_processed.drop('id', axis=1)
cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_processed[col] = df_processed[col].map({'Yes': 1, 'No': 0})
le = LabelEncoder()
df_processed['Personality'] = le.fit_transform(df_processed['Personality'])

# Separar características (X) y objetivo (y) antes de imputar
X = df_processed.drop('Personality', axis=1)
y = df_processed['Personality']

# Guardamos nombres de columnas e índice para reconstruir el DataFrame
X_columns = X.columns
X_index = X.index

# Entrenar (fit) y transformar el imputer SOLO en las características X
print("Iniciando imputación con IterativeImputer...")
imputer = IterativeImputer(max_iter=10, random_state=42)
X_imputed_array = imputer.fit_transform(X)

# Reconstruir el DataFrame de características X
X_processed = pd.DataFrame(X_imputed_array, columns=X_columns, index=X_index)
print("¡Imputación completada!")

Iniciando imputación con IterativeImputer...
¡Imputación completada!


C:\Users\fabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### 1. Re-evaluación del Modelo Base con Imputación Avanzada

In [ ]:
# Separar los datos en características (X) y objetivo (y)
X = df_processed.drop('Personality', axis=1)
y = df_processed['Personality']

# Dividir los datos en conjuntos de entrenamiento y prueba
# Usamos stratify=y para asegurar que la proporción de introvertidos/extrovertidos sea la misma en ambos conjuntos, lo cual es crucial por el desbalance.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Datos divididos en entrenamiento y prueba.")
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")

# Crear y entrenar el modelo Random Forest
# Usamos class_weight='balanced' para que el modelo preste más atención a la clase minoritaria (introvertidos) y no se deje llevar por la mayoría.
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

print("\nEntrenando el modelo Random Forest...")
rf_model.fit(X_train, y_train)
print("¡Modelo entrenado!")

# Hacer predicciones y evaluar el modelo
print("\nEvaluando el modelo en el conjunto de prueba...")
y_pred = rf_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy (Precisión) del modelo: {accuracy:.4f}")

# Mostrar el reporte de clasificación completo con precision, recall y f1-score para cada clase.
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Extrovert', 'Introvert']))

Datos divididos en entrenamiento y prueba.
Tamaño de X_train: (14819, 7)
Tamaño de X_test: (3705, 7)

Entrenando el modelo Random Forest...
¡Modelo entrenado!

Evaluando el modelo en el conjunto de prueba...

Accuracy (Precisión) del modelo: 0.9695

Reporte de Clasificación:
              precision    recall  f1-score   support

   Extrovert       0.98      0.98      0.98      2740
   Introvert       0.94      0.94      0.94       965

    accuracy                           0.97      3705
   macro avg       0.96      0.96      0.96      3705
weighted avg       0.97      0.97      0.97      3705



In [17]:
# Definir la parrilla de parámetros a probar
param_grid = {
    'n_estimators': [100, 200],         # Número de árboles en el bosque
    'max_depth': [10, 20, None],        # Profundidad máxima de los árboles
    'min_samples_leaf': [1, 2, 4]       # Mínimo de muestras en un nodo hoja
}

# Configurar GridSearchCV
# cv=5 significa validación cruzada de 5 folds
# n_jobs=-1 usa todos los procesadores para ir más rápido
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

print("Iniciando GridSearchCV... Esto puede tardar varios minutos.")
# Entrenamos la búsqueda sobre el conjunto de entrenamiento completo
grid_search.fit(X_train, y_train)

print("\nGridSearchCV completado.")
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor score de cross-validation (accuracy): {grid_search.best_score_:.4f}")

Iniciando GridSearchCV... Esto puede tardar varios minutos.
Fitting 5 folds for each of 18 candidates, totalling 90 fits

GridSearchCV completado.
Mejores parámetros encontrados: {'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 100}
Mejor score de cross-validation (accuracy): 0.9682


In [ ]:
# Cargar el dataset de test original
df_test = pd.read_csv('test.csv')
# Guardar los IDs para el archivo de submission
test_ids = df_test['id']

# Aplicar el mismo preprocesamiento al test set
df_test_processed = df_test.copy()
df_test_processed = df_test_processed.drop('id', axis=1)

# Convertir Stage_fear 
df_test_processed['Stage_fear'] = pd.to_numeric(df_test_processed['Stage_fear'], errors='coerce')

# Mapear columnas categóricas
cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_test_processed[col] = df_test_processed[col].map({'Yes': 1, 'No': 0})

# ENTRENAR MODELO FINAL Y PREDECIR 

# Crear el modelo final con los mejores parámetros de GridSearchCV
final_model = RandomForestClassifier(random_state=42, **grid_search.best_params_)

# Entrenar con todos los datos de entrenamiento
final_model.fit(X, y)

# Predecir sobre el test set procesado
test_predictions = final_model.predict(df_test_processed)

# Invertir la codificación para tener 'Introvert'/'Extrovert'
final_predictions_labels = le.inverse_transform(test_predictions.astype(int))

# Archivo de submission
submission_df = pd.DataFrame({'id': test_ids, 'Personality': final_predictions_labels})
submission_df.to_csv('submission2.csv', index=False)

print("\nArchivo 'submission2.csv' creado exitosamente.")


Archivo 'submission2.csv' creado exitosamente. ¡Listo para subir a Kaggle!


#### `Kaggle Score: 0.975708`

### 2. Incorporación de Modelos de Gradient Boosting (XGBoost y LightGBM)


In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Modelo XGBoost
xgb_model = XGBClassifier(random_state=42, objective='binary:logistic', eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Modelo LightGBM
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [ ]:
from sklearn.ensemble import VotingClassifier

# Recupera tu mejor modelo de Random Forest
best_rf = grid_search.best_estimator_

# Crea el ensamblador 'soft' promedia las probabilidades, lo que suele dar mejores resultados
voting_clf = VotingClassifier(
    estimators=[('rf', best_rf), ('xgb', xgb_model), ('lgbm', lgbm_model)],
    voting='soft'
)

# Entrena el modelo ensamblado
print("Entrenando el modelo de ensamblado...")
voting_clf.fit(X_train, y_train)
print("¡Ensamblado entrenado!")

# Evalúa el ensamblado
y_pred_ensemble = voting_clf.predict(X_test)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f"\nAccuracy del Ensamble: {accuracy_ensemble:.4f}")

Entrenando el modelo de ensamblado...
[LightGBM] [Info] Number of positive: 3860, number of negative: 10959
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000335 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.260476 -> initscore=-1.043494
[LightGBM] [Info] Start training from score -1.043494
¡Ensamblado entrenado!

Accuracy del Ensamble: 0.9711


In [ ]:
# Cargar y procesar el test set
df_test = pd.read_csv('test.csv')
test_ids = df_test['id']
df_test_processed = df_test.copy()
df_test_processed = df_test_processed.drop('id', axis=1)

cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_test_processed[col] = df_test_processed[col].map({'Yes': 1, 'No': 0})

test_columns = df_test_processed.columns
test_index = df_test_processed.index

# Aplicar el imputer ya entrenado (SOLO .transform())
print("Aplicando IterativeImputer a los datos de prueba...")
df_test_imputed_array = imputer.transform(df_test_processed)
df_test_processed = pd.DataFrame(df_test_imputed_array, columns=test_columns, index=test_index)
print("¡Imputación del test set completada!")

# Predecir sobre el test set procesado
print("\n🚀 Realizando predicciones...")
final_predictions = voting_clf.predict(df_test_processed) # Usa tu mejor modelo aquí
final_predictions_labels = le.inverse_transform(final_predictions.astype(int))

# Archivo de submission
submission_df = pd.DataFrame({'id': test_ids, 'Personality': final_predictions_labels})
submission_df.to_csv('submission_final.csv', index=False)
print("\nArchivo 'submission_final.csv' creado exitosamente.")

Aplicando IterativeImputer a los datos de prueba...
¡Imputación del test set completada!

🚀 Realizando predicciones...

Archivo 'submission_final_corregida.csv' creado exitosamente.


#### `Kaggle Score: 0.974898`

### Optimizacion del Gradient Boosting con Optuna

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# Definir la función "objective" para XGBoost
def objective_xgb(trial):
    # Definimos el espacio de búsqueda de hiperparámetros para XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42
    }

    # Creamos el modelo con los parámetros del trial
    model = XGBClassifier(**params)

    # Evaluamos con validación cruzada
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=5, scoring='accuracy')
    accuracy = score.mean()
    
    return accuracy

# Crear y ejecutar el estudio de Optuna
study_xgb = optuna.create_study(direction='maximize')
print("🚀 Iniciando optimización de XGBoost con Optuna...")
study_xgb.optimize(objective_xgb, n_trials=50) # Puedes aumentar los trials para una búsqueda más exhaustiva

# Mostrar los resultados
print("\nOptimización de XGBoost completada.")
print(f"Mejor score de cross-validation (accuracy): {study_xgb.best_value:.4f}")
print("Mejores parámetros encontrados para XGBoost:")
print(study_xgb.best_params)

C:\Users\fabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-28 20:23:39,112] A new study created in memory with name: no-name-1d45d8af-4a63-4794-8592-68e216ed8c70


🚀 Iniciando optimización de XGBoost con Optuna...


[I 2025-07-28 20:23:43,167] Trial 0 finished with value: 0.9663270074508684 and parameters: {'n_estimators': 521, 'max_depth': 4, 'learning_rate': 0.2937847328847428, 'subsample': 0.9133131250423491, 'colsample_bytree': 0.6054156447778489, 'gamma': 0.28053146166103715}. Best is trial 0 with value: 0.9663270074508684.
[I 2025-07-28 20:23:47,528] Trial 1 finished with value: 0.9611310298904664 and parameters: {'n_estimators': 908, 'max_depth': 8, 'learning_rate': 0.11778539036279655, 'subsample': 0.6601042930984006, 'colsample_bytree': 0.6215445966590865, 'gamma': 0.1493176287828729}. Best is trial 0 with value: 0.9663270074508684.
[I 2025-07-28 20:23:49,902] Trial 2 finished with value: 0.9676765123431907 and parameters: {'n_estimators': 404, 'max_depth': 10, 'learning_rate': 0.18289430504767892, 'subsample': 0.7583523237600015, 'colsample_bytree': 0.6782848969774726, 'gamma': 1.4233196687988197}. Best is trial 2 with value: 0.9676765123431907.
[I 2025-07-28 20:23:50,598] Trial 3 finish


Optimización de XGBoost completada.
Mejor score de cross-validation (accuracy): 0.9688
Mejores parámetros encontrados para XGBoost:
{'n_estimators': 793, 'max_depth': 10, 'learning_rate': 0.17306544173662783, 'subsample': 0.9441961486485528, 'colsample_bytree': 0.7760704908880952, 'gamma': 4.5659745128683005}


### 3. Creación de un Ensamble Definitivo con Modelos Optimizados



In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Crear tus dos modelos campeones con sus mejores parámetros
best_rf_params = {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 200} # De tu GridSearchCV
best_xgb_params = study_xgb.best_params # De tu estudio con Optuna

# Instanciamos los modelos
champion_rf = RandomForestClassifier(random_state=42, class_weight='balanced', **best_rf_params)
champion_xgb = XGBClassifier(random_state=42, **best_xgb_params)


# Crear el VotingClassifier final. Usamos 'soft' voting para promediar las probabilidades, que es más robusto
final_ensemble = VotingClassifier(
    estimators=[('rf', champion_rf), ('xgb', champion_xgb)],
    voting='soft'
)

# Entrenar el ensamblado con TODOS los datos de entrenamiento
print("Entrenando el ensamble definitivo...")
# Recuerda usar X_processed, que son tus datos imputados con IterativeImputer
final_ensemble.fit(X_processed, y)
print("¡Modelo final entrenado!")


# Procesar el test set y generar la submission (código ya validado)
df_test = pd.read_csv('test.csv')
test_ids = df_test['id']
df_test_processed = df_test.copy()
df_test_processed = df_test_processed.drop('id', axis=1)
cols_to_map = ['Stage_fear', 'Drained_after_socializing']
for col in cols_to_map:
    df_test_processed[col] = df_test_processed[col].map({'Yes': 1, 'No': 0})
test_columns = df_test_processed.columns
test_index = df_test_processed.index

# Aplicamos el imputer ya entrenado
df_test_imputed_array = imputer.transform(df_test_processed)
df_test_processed = pd.DataFrame(df_test_imputed_array, columns=test_columns, index=test_index)


# Predecir y crear el archivo final
print("\nHaciendo predicciones finales...")
final_predictions = final_ensemble.predict(df_test_processed)
final_predictions_labels = le.inverse_transform(final_predictions.astype(int))

submission_df = pd.DataFrame({'id': test_ids, 'Personality': final_predictions_labels})
submission_df.to_csv('submission_ultimate_ensemble.csv', index=False)

print("\nArchivo 'submission_ultimate_ensemble.csv' creado. ¡Listo para tu envío final a Kaggle!")

🚀 Entrenando el ensamble definitivo...
¡Modelo final entrenado!

Haciendo predicciones finales...

✅ Archivo 'submission_ultimate_ensemble.csv' creado. ¡Listo para tu envío final a Kaggle!


#### `Kaggle Score: 0.95708`

#### Conclusión
Tras una exploración exhaustiva con técnicas avanzadas, se logró una mejora en el rendimiento del modelo.

#### Resumen de Resultados
El uso de **`IterativeImputer`** demostró ser beneficioso, permitiendo que los modelos alcanzaran un rendimiento ligeramente superior en la validación local.

El modelo **`RandomForestClassifier`**, optimizado con los hiperparámetros optimizados por **Optuna**, fue el que obtuvo el mejor rendimiento individual.

#### El score final más alto en Kaggle fue de **`0.975708`**, logrado con el RandomForestClassifier entrenado sobre los datos procesados con IterativeImputer.

#### Conclusión General
El proyecto demuestra que, si bien una base sólida con técnicas estándar puede lograr resultados excelentes (Score: 0.973), la aplicación de métodos más avanzados como IterativeImputer puede proporcionar una mejora marginal pero decisiva. Se concluye que el RandomForestClassifier optimizado es el modelo campeón para este problema, confirmando que se ha alcanzado el máximo potencial predictivo de los datos.